In [1]:
import matplotlib.pyplot as plt
import numpy as np
import PIL
import os
import tensorflow as tf
from tensorflow.keras.optimizers import Adam
import math
from tqdm import tqdm
from sklearn.model_selection import train_test_split
from vae_vanilla import VAE

In [2]:
## Loading dataset
labels = np.load('./data/LFSD_labels.npy')
depths = np.load('./data/LFSD_depths_repeated.npy')
imgs = np.load('./data/LFSD_imgs.npy')
masks = np.load('./data/LFSD_masks_single.npy')

def built_batch_iterator(num_data, batch_size):
    idx = np.random.permutation(num_data)
    return [idx[i:i + batch_size] for i in range(0, num_data, batch_size)]

batch_idxs = built_batch_iterator(len(labels), 64)
dataset = []
for idx in batch_idxs:
    img_batch, depth_batch, mask_batch = imgs[idx], depths[idx], masks[idx]
    dataset.append((img_batch, depth_batch, mask_batch))
train_dataset, test_dataset = train_test_split(dataset, test_size=0.3)

In [3]:
latent_dim = 2048
vae = VAE(latent_dim)
learning_rate = 1e-4
vae.compile(optimizer=Adam(learning_rate))
epochs = 10

def merge_images(image_batch, size):
    h,w = image_batch.shape[1], image_batch.shape[2]
    img = np.zeros((int(h*size[0]), w*size[1]))
    for idx, im in enumerate(image_batch):
        im = np.squeeze(im, axis=2)
        i = idx % size[1]
        j = idx // size[1]
        img[j*h:j*h+h, i*w:i*w+w] = im
    return img

losses_across_epochs = {
    "loss": [],
    "reconstruction_loss": [],
    "kl_loss": [],
}
batch_num = len(train_dataset)
for i in range(epochs):
    print("Epoch %d: " % i)
    for k, v in losses_across_epochs.items():
        losses_across_epochs[k].append(0)
    for data in train_dataset:
        cur_loss = vae.train_step(data)
        for k, v in cur_loss.items():
            losses_across_epochs[k][-1] += cur_loss[k].numpy() / batch_num
        generated_image = vae.sample(data)
    print("Total loss: %d" % losses_across_epochs['loss'][-1])
    im_merged = merge_images(generated_image.numpy(), [8,8])
    plt.imsave('./images/vae_vanilla/%d.png' % i, im_merged, cmap='gray')
for k, v in losses_across_epochs.items():
    np.save('./results/vae_vanilla/%s' % k, np.array(v))
vae.save_weights('./trained_weights/vae_vanilla_inception')

Epoch 0: 
Total loss: 2910944
Epoch 1: 
Total loss: 2899735
Epoch 2: 
Total loss: 2847573
Epoch 3: 
Total loss: 2740655
Epoch 4: 
Total loss: 2599842
Epoch 5: 
Total loss: 2445587
Epoch 6: 
Total loss: 2290334
Epoch 7: 
Total loss: 2148266
Epoch 8: 
Total loss: 2021396
Epoch 9: 
Total loss: 1906488


In [5]:
test_loss = 0
for i, data in enumerate(test_dataset):
    _, _, mask_batch = data
    generated_image = vae.sample(data)
    reconstruction_loss = tf.reduce_sum(
        tf.keras.losses.binary_crossentropy(mask_batch, generated_image), [1,2]
    )
    test_loss += tf.reduce_mean(reconstruction_loss).numpy()
    im_merged = merge_images(generated_image.numpy(), [8,8])
    plt.imsave('./images/vae_vanilla/test_batch_%d.png' % i, im_merged, cmap='gray')
    
test_loss = test_loss / len(test_dataset)
np.save('./results/vae_vanilla/test_loss', np.array([test_loss]))

In [18]:
from skimage.transform import resize
from tensorflow.keras import datasets

train_dataset, test_dataset = None, None

(train_images, train_labels), (test_images, test_labels) = datasets.cifar100.load_data()
result = []

for img in tqdm(train_images):
    img = np.expand_dims(resize(img, (256, 256, 3)), 0)
    activation, _, _ = vae.encode(img, None, rand_depth=True)
    activation = np.squeeze(activation, axis=0)
    result.append(activation)

np.save('./data/CIFAR100_vae_vanilla_encoding_train.npy', np.array(result))

  6%|▌         | 2762/50000 [01:04<18:22, 42.83it/s]


KeyboardInterrupt: 

In [19]:
result = []
for img in tqdm(test_images):
    img = np.expand_dims(resize(img, (256, 256, 3)), 0)
    activation, _, _ = new_vae.encode(img, None, rand_depth=True)
    activation = np.squeeze(activation, axis=0)
    result.append(activation)
np.save('./data/CIFAR100_vae_vanilla_encoding_test.npy', np.array(result))

  2%|▏         | 157/10000 [00:03<03:59, 41.05it/s]


KeyboardInterrupt: 